In [56]:
import numpy as np
from scipy.optimize import minimize

W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([10000000, 450002])

def f(r):
    return ((r-1)**2)

def objective(W):
    return (1/2)*sum(
        d_k*f(w_k, d_k) for w_k, d_k in zip(W, D)
    )

def constraint(W):
    return A.T@W-B

constraints = {"type" : "eq", "fun" : constraint}

result = minimize(objective, 
                  x0=x0, 
                  method="trust-constr", 
                  constraints=constraints)

In [57]:
result

          message: `xtol` termination condition is satisfied.
          success: True
           status: 2
              fun: 7982207.6027833745
                x: [ 1.384e+04 -1.293e+02 -1.614e+04 -3.372e+04]
              nit: 29
             nfev: 225
             njev: 45
             nhev: 0
         cg_niter: 30
     cg_stop_cond: 4
             grad: [ 1.374e+02 -2.293e+00 -1.624e+02 -3.382e+02]
  lagrangian_grad: [ 3.281e-07  3.943e-04  2.875e-06 -3.922e-06]
           constr: [array([-1.863e-09,  0.000e+00])]
              jac: [array([[ 1.000e+03, -0.000e+00,  2.500e+01,  1.020e+02],
                          [ 2.584e+03,  4.001e+00,  3.540e+02,  8.780e+02]])]
      constr_nfev: [225]
      constr_njev: [0]
      constr_nhev: [0]
                v: [array([-1.619e+00,  5.732e-01])]
           method: equality_constrained_sqp
       optimality: 0.0003942959068741203
 constr_violation: 1.862645149230957e-09
   execution_time: 0.07196497917175293
        tr_radius: 1.72709484988